In [48]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3

In [52]:
import os
os.chdir("/home/sunil/Desktop/boww2v")
print (os.getcwd())

/home/sunil/Desktop/boww2v


In [4]:
from bs4 import BeautifulSoup
#import two lists: titles, and synopses
titles = open('title_list.txt').read().split('\n')
#ensures that only the first 100 are read in
titles = titles[:100]
synopses_imdb = open('synopses_list_imdb.txt').read().split('\n BREAKS HERE')
synopses_imdb = synopses_imdb[:100]

synopses_clean_imdb = []

for text in synopses_imdb:
    text = BeautifulSoup(text, 'html.parser').getText()
    #strips html formatting and converts to unicode
    synopses_clean_imdb.append(text)

synopses_imdb = synopses_clean_imdb
synopses = []

for i in range(len(synopses_imdb)):
    item = synopses_imdb[i]
    synopses.append(item)

In [5]:
print len(synopses)
print len(titles)

100
100


In [12]:
print titles[2]

Schindler's List


In [13]:
print synopses[2]



The relocation of Polish Jews from surrounding areas to Krakow begins in late 1939, shortly after the outbreak of World War II, when the German Army defeats the Polish Army in three weeks. Oskar Schindler (Liam Neeson), a successful businessman, arrives from Czechoslovakia in hopes of using the abundant cheap labour force of Jews to manufacture enamelware for the German military. Schindler, an opportunistic member of the Nazi party, lavishes bribes upon the army and SS officials in charge of procurement. Sponsored by the military, Schindler acquires a factory for the production of army mess kits and cooking paraphernalia. Not knowing much about how to properly run such an enterprise, he gains a contact in Itzhak Stern (Ben Kingsley), a functionary in the local Judenrat (Jewish Council) who has contacts with the now-underground Jewish business community in the ghetto. They loan him the money for the factory in return for a small share of products produced (for trade on the black marke

In [14]:
print titles[:10]

['The Godfather', 'The Shawshank Redemption', "Schindler's List", 'Raging Bull', 'Casablanca', "One Flew Over the Cuckoo's Nest", 'Gone with the Wind', 'Citizen Kane', 'The Wizard of Oz', 'Titanic']


In [15]:
print synopses[0][:200]



In late summer 1945, guests are gathered for the wedding reception of Don Vito Corleone's daughter Connie (Talia Shire) and Carlo Rizzi (Gianni Russo). Vito (Marlon Brando), the head of the Corleone


In [16]:
stopwords = nltk.corpus.stopwords.words('english')

In [17]:
print stopwords[:10]

[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your']


In [18]:
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [19]:
# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [20]:
#use extend so it's a big flat list of vocab
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in synopses:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [25]:
totalvocab_stemmed[10]

u'of'

In [26]:
totalvocab_tokenized[10]

u'of'

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(synopses) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

CPU times: user 5.17 s, sys: 64 ms, total: 5.24 s
Wall time: 5.19 s
(100, 372)


In [28]:
terms = tfidf_vectorizer.get_feature_names()


In [29]:
print terms

[u"'d", u'abl', u'accept', u'accompani', u'act', u'action', u'admit', u'afterward', u'agre', u'air', u'aliv', u'allow', u'alon', u'alreadi', u'alway', u'american', u'ani', u'anoth', u'anyon', u'anyth', u'apart', u'appar', u'appear', u'approach', u'area', u'arm', u'armi', u'arrang', u'arrest', u'arriv', u'ask', u'attack', u'attempt', u'attend', u'avoid', u'away', u'bad', u'bar', u'battl', u'beat', u'becaus', u'becom', u'bed', u'befor', u'begin', u'believ', u'better', u'big', u'black', u'board', u'bodi', u'boy', u'break', u'bring', u'brother', u'build', u'busi', u'buy', u'ca', u"ca n't", u'car', u'care', u'carri', u'catch', u'caus', u'chang', u'charg', u'chase', u'child', u'children', u'citi', u'claim', u'clear', u'close', u'cold', u'come', u'command', u'complet', u'confront', u'consid', u'continu', u'convers', u'convinc', u'coupl', u'cover', u'cross', u'cut', u'dark', u'daughter', u'day', u'dead', u'death', u'decid', u'demand', u'despit', u'destroy', u'did', u'die', u'direct', u'discov'

In [32]:
from sklearn.metrics.pairwise import cosine_similarity
dist = cosine_similarity(tfidf_matrix)
print dist

[[ 1.          0.26015693  0.25786319 ...,  0.41102123  0.40919079  0.        ]
 [ 0.26015693  1.          0.31190325 ...,  0.35272122  0.32446419  0.        ]
 [ 0.25786319  0.31190325  1.         ...,  0.28976346  0.29452701  0.        ]
 ..., 
 [ 0.41102123  0.35272122  0.28976346 ...,  1.          0.66208677  0.        ]
 [ 0.40919079  0.32446419  0.29452701 ...,  0.66208677  1.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]]


In [35]:
dist[8]

array([ 0.27120302,  0.27773673,  0.2163732 ,  0.13939002,  0.32275932,
        0.32806528,  0.37195595,  0.26906931,  1.        ,  0.15166647,
        0.28129162,  0.29265742,  0.33047318,  0.26323399,  0.34501623,
        0.40822904,  0.2678433 ,  0.18945155,  0.31909933,  0.41608597,
        0.40999747,  0.28143208,  0.2403546 ,  0.2957111 ,  0.20575076,
        0.2072054 ,  0.35695361,  0.32661985,  0.10840836,  0.17217577,
        0.17910854,  0.19500568,  0.22442074,  0.32378487,  0.2442254 ,
        0.27894465,  0.30017245,  0.33489245,  0.36224329,  0.4133192 ,
        0.22312929,  0.19996767,  0.2861762 ,  0.12256007,  0.33282564,
        0.22749002,  0.33995988,  0.27549229,  0.18358597,  0.32490451,
        0.21754671,  0.18598302,  0.27640809,  0.25023286,  0.15603483,
        0.21976259,  0.        ,  0.18525404,  0.31209715,  0.22609957,
        0.32282219,  0.27292247,  0.        ,  0.27079668,  0.        ,
        0.21979581,  0.10457177,  0.26800157,  0.22924921,  0.29

In [36]:
from sklearn.cluster import KMeans

num_clusters = 5

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

CPU times: user 392 ms, sys: 4 ms, total: 396 ms
Wall time: 394 ms


In [39]:
print clusters

[0, 1, 1, 0, 1, 1, 4, 4, 2, 4, 1, 0, 4, 4, 4, 4, 4, 2, 4, 1, 2, 4, 4, 4, 1, 3, 4, 4, 3, 1, 4, 1, 4, 1, 0, 1, 1, 1, 1, 4, 0, 0, 4, 0, 0, 2, 2, 0, 1, 4, 0, 1, 1, 2, 4, 1, 3, 1, 0, 0, 4, 1, 3, 4, 3, 0, 4, 4, 2, 2, 4, 4, 0, 4, 3, 4, 4, 0, 0, 3, 1, 4, 3, 4, 4, 4, 4, 3, 3, 3, 4, 4, 4, 3, 4, 4, 4, 4, 4, 3]


In [40]:
films = { 'title': titles, 'synopsis': synopses, 'cluster': clusters}

frame = pd.DataFrame(films, index = [clusters] , columns = ['title', 'cluster'])

In [41]:
frame['cluster'].value_counts()

4    42
1    21
0    16
3    13
2     8
Name: cluster, dtype: int64